In [1]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
   # Request headers
   'Content-Type': 'application/json',
   'Prediction-key': '009281becef44f2c85dd96cd749d56ea',
}

params = urllib.parse.urlencode({
   # Request parameters
   'iterationId': '22f47df0-85a5-42e2-9e0c-d73a1591ad1d',
   'application': 'team 12 hardshell',
})

try:
   conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
   conn.request("POST", "/customvision/v2.0/Prediction/dbb69137-acbf-4209-88dd-3220c4011eb3/url?%s" % params, '{"Url": "http://img.cdn.mountainwarehouse.com/product/025374/025374_pur_featherweight_down_womens_jacket__wms_aw17_2.jpg"}', headers)
   response = conn.getresponse()
   data = response.read()
   print(data)
   conn.close()
except Exception as e:
   print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"id":"03935125-878a-4c88-83ff-5a9a64d6a7ce","project":"e6da02d6-eac5-475e-b891-1e2400519a69","iteration":"22f47df0-85a5-42e2-9e0c-d73a1591ad1d","created":"2018-05-23T15:20:14.0810079Z","predictions":[{"probability":0.99997884,"tagId":"0b1f5b9b-038e-4eec-b20d-133850f2edd1","tagName":"insulated jacket"},{"probability":1.55497819E-05,"tagId":"bb3320bc-d247-4c0a-8cd9-3355a7b30c6a","tagName":"hardshell jacket"}]}'


In [ ]:
# USAGE
# Start the server:
# 	python run_keras_server.py
# Submit a request via cURL:
# 	curl -X POST -F image=@dog.jpg 'http://localhost:5000/predict'
# Submita a request via Python:
#	python simple_request.py

# import the necessary packages
from keras.applications import ResNet50
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.models import load_model
from PIL import Image
import numpy as np
import flask
import io

# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
model = None
lookup = {0: 'axes', 1: 'boots', 2: 'carabiners', 3: 'crampons', 4: 'gloves', 5: 'hardshell_jackets',
          6: 'harnesses', 7: 'helmets', 8: 'insulated_jackets', 9: 'pulleys', 10: 'rope', 11: 'tents'}


def load_model_s():
    # load the pre-trained Keras model (here we are using a model
    # pre-trained on ImageNet and provided by Keras, but you can
    # substitute in your own networks just as easily)
    global model
    #model = ResNet50(weights="imagenet")
    model = load_model('cnn-aw-model.h5')


def prepare_image(image, target):
    # if the image mode is not RGB, convert it
    if image.mode != "RGB":
        image = image.convert("RGB")

    # resize the input image and preprocess it
    image = image.resize(target)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = imagenet_utils.preprocess_input(image)

    # return the processed image
    return image


@app.route("/predict", methods=["POST"])
def predict():
    # initialize the data dictionary that will be returned from the
    # view
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    if flask.request.method == "POST":
        if flask.request.files.get("image"):
            # read the image in PIL format
            image = flask.request.files["image"].read()
            image = Image.open(io.BytesIO(image))

            # preprocess the image and prepare it for classification
            image = prepare_image(image, target=(128, 128))

            # classify the input image and then initialize the list
            # of predictions to return to the client
            preds = model.predict(image)[0]
            #results = imagenet_utils.decode_predictions(preds)
            #preds = preds.astype('uint8')
            print(preds)
            data["predictions"] = np.array_str(preds)

            # loop over the results and add them to the list of
            # returned predictions
            # for (imagenetID, label, prob) in preds[0]:
            # 		r = {"label": label, "probability": float(prob)}
     #		data["predictions"].append(r)

            # indicate that the request was a success
            data["success"] = True

    # return the data dictionary as a JSON response
    return flask.jsonify(data)


# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
           "please wait until server has fully started"))
    load_model_s()
    app.run(host='0.0.0.0')
